In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import joblib
from playstore.config import CLEAN_DATA_DIR

In [2]:
# Load data
try:
    df = pd.read_csv("cleaned_googleplaystore.csv")  # Update to your actual file path if different
    print("Loaded")
except Exception as e:
    print("Error loading data:", e)
    df = pd.DataFrame({
        'Category': ['Game', 'Education', 'Business'],
        'Rating': [4.5, 4.0, 3.8],
        'Type': ['Free', 'Free', 'Paid'],
        'Installs': [1000000, 500000, 100000],
    })


Loaded


In [4]:

# Select numeric columns and drop NaN values
numeric_columns = ['Reviews', 'Size', 'Installs', 'Price', 'Updated_Day', 'Updated_Month', 'Updated_Year', 'Rating']
dfnum = df[numeric_columns].dropna()

# Handle outliers
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.10)
    quartile3 = dataframe[variable].quantile(0.90)
    interquartile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquartile_range
    low_limit = quartile1 - 1.5 * interquartile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, numeric_columns):
    for variable in numeric_columns:
        if dataframe[variable].dtype.kind in 'biufc':  # Numeric columns only
            low_limit, up_limit = outlier_thresholds(dataframe, variable)
            dataframe.loc[dataframe[variable] < low_limit, variable] = low_limit
            dataframe.loc[dataframe[variable] > up_limit, variable] = up_limit

replace_with_thresholds(dfnum, dfnum.columns)

# Prepare data for training
y = dfnum["Rating"]  # Target variable
x = dfnum.drop(columns=["Rating"], errors="ignore")  # Features

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.81, random_state=81)

# Train the Random Forest model
rf = RandomForestRegressor(random_state=81)
model = rf.fit(x_train, y_train)

# Evaluate the model
score = model.score(x_test, y_test)
print("Model Score:", score)

# Save the model
model_path = os.path.join(os.getcwd(), "rf_model4.pkl")
joblib.dump(model, model_path)
print(f"Model saved to {model_path}")


/var/folders/kd/bldn4b0j3q31pb8c56pgmyy80000gn/T/ipykernel_61227/33551356.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-823644.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataframe.loc[dataframe[variable] < low_limit, variable] = low_limit
/var/folders/kd/bldn4b0j3q31pb8c56pgmyy80000gn/T/ipykernel_61227/33551356.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-7.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataframe.loc[dataframe[variable] < low_limit, variable] = low_limit


Model Score: 0.10400870776621507
Model saved to /Users/steve/PycharmProjects/uu_school/playstore/notebooks/rf_model4.pkl
